In [5]:
import traceback
import os
import time
import requests
import numpy as np
import pandas as pd
import pandas_ta
# import yfinance as yf # Not strictly used if Twelve Data is primary
from datetime import datetime, timedelta
import pywt
import antropy as ant
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import TimeSeriesSplit # Not directly used in main flow, Optuna handles its split.
from sklearn.metrics import accuracy_score, classification_report, log_loss, roc_auc_score
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest, mutual_info_classif, VarianceThreshold
# import gc # Unused

# Foundational Model Imports
from transformers import PatchTSTConfig, PatchTSTForPrediction # Removed Trainer, TrainingArguments, EarlyStoppingCallback, set_seed

# --- Global Flags and Initializations for Optional Libraries ---
optuna_available = False
torch_available = False
# sktime_available = False # Becomes irrelevant if add_clasp_regimes is removed
# ClaSPSegmentation = None # Becomes irrelevant
dowhy_available = False
CausalModel = None
nx = None
# ONNX related flags and types
onnx_available = False
skl2onnx_available = False
onnxmltools_available = False
FloatTensorType = None # Crucial to initialize globally for the check later
SKIP_CAUSAL_ANALYSIS_FOR_DEBUGGING = True # Set to False to enable Causal Analysis

if hasattr(pd.DataFrame, 'ta') is False and pandas_ta is not None:
    try:
        pd.DataFrame.ta = pandas_ta.Core(df=None)
        print("pandas_ta DataFrame accessor registered globally.")
    except Exception as e:
        print(f"Could not globally register pandas_ta accessor: {e}")

try:
    import optuna
    optuna_available = True
    print("Optuna imported successfully.")
except ImportError:
    print("Optuna not found. LightGBM hyperparameter optimization with Optuna will be skipped.")

try:
    import torch
    import torch.nn as nn
    import torch.optim as optim
    torch_available = True
    print("PyTorch imported successfully.")
    if torch.cuda.is_available():
        print(f"PyTorch CUDA available: True, Version: {torch.version.cuda}")
        print(f"Using PyTorch on GPU: {torch.cuda.get_device_name(0)}")
    else:
        print("PyTorch CUDA available: False.")
except ImportError:
    print("PyTorch not found. Custom Transformer and Foundation Model features will be SKIPPED.")

try:
    import dowhy
    from dowhy import CausalModel
    import networkx as nx # networkx is often used with dowhy
    dowhy_available = True
    print(f"DoWhy {dowhy.__version__} and NetworkX {nx.__version__} imported successfully.")
except ImportError:
    print("DoWhy or NetworkX not found. Causal Discovery will be skipped.")

try:
    import onnx
    onnx_available = True
    # import onnxruntime as ort
    import skl2onnx
    skl2onnx_available = True
    from skl2onnx.common.data_types import FloatTensorType # Assigns to global FloatTensorType
    import onnxmltools
    onnxmltools_available = True
    print("ONNX, ONNXRuntime, skl2onnx, and onnxmltools imported successfully.")
    if hasattr(onnxmltools, '__version__'):
          print(f"Onnxmltools version: {onnxmltools.__version__}")
except ImportError as e:
    print(f"One or more ONNX components not found: {e}. ONNX features will be skipped.")

print("\nAll libraries and modules conditional imports attempted.")

# --- Constants ---
TWELVE_DATA_API_KEY = "b6dbb92e551a46f2b20de27540aeef0a" # Replace with your actual key
API_KEY = TWELVE_DATA_API_KEY # Ensure this is set
DEFAULT_SYMBOL = "MSFT"
START_DATE = (datetime.now() - timedelta(days=3*365)).strftime('%Y-%m-%d')
END_DATE = datetime.now().strftime('%Y-%m-%d')
AUTOFORMER_MODEL_PATH = "autoformer_predictor_trained.pth" # For optional Autoformer

# --- Function Definitions (AutoformerPredictor and its helpers - Conditionally Used) ---
class AutoformerPredictor:
    def __init__(self, input_len=60, pred_len=5, d_model=64, n_heads=8, num_encoder_layers=2, num_decoder_layers=1, dim_feedforward=256):
        self.device = 'cuda' if torch.cuda.is_available() and torch_available else 'cpu'
        self.model = None
        self.input_len = input_len
        self.pred_len = pred_len
        self.d_model = d_model

        if torch_available:
            # print(f"🔧 Initializing Custom AutoformerPredictor on device: {self.device}") # Less verbose
            self.model = nn.Transformer(
                d_model=d_model, nhead=n_heads, num_encoder_layers=num_encoder_layers,
                num_decoder_layers=num_decoder_layers, dim_feedforward=dim_feedforward,
                activation='gelu', batch_first=False #Transformer expects seq_len first by default
            ).to(self.device)
            self.enc_embedding = nn.Linear(1, d_model).to(self.device)
            self.dec_embedding = nn.Linear(1, d_model).to(self.device)
            self.projection = nn.Linear(d_model, 1).to(self.device)
        else:
            print("PyTorch not available. AutoformerPredictor cannot be initialized.")

    def _generate_square_subsequent_mask(self, sz):
        if not torch_available: return None
        mask = (torch.triu(torch.ones(sz, sz, device=self.device)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def forward(self, src, tgt): # src: (seq_len, batch, features), tgt: (tgt_len, batch, features)
        if not torch_available or self.model is None: return None
        src_embedded = self.enc_embedding(src)
        tgt_embedded = self.dec_embedding(tgt)
        tgt_mask = self._generate_square_subsequent_mask(tgt.size(0))
        output = self.model(src_embedded, tgt_embedded, tgt_mask=tgt_mask)
        output = self.projection(output)
        return output

    def train_model(self, train_loader, epochs=10, learning_rate=0.001):
        if not torch_available or self.model is None: print("Cannot train: PyTorch or model not available."); return
        if not train_loader: print("Cannot train: No data loader provided."); return
        print(f"Starting AutoformerPredictor training for {epochs} epochs...")
        self.model.train()
        optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)
        criterion = nn.MSELoss()
        for epoch in range(epochs):
            epoch_loss = 0
            for i, (batch_src, batch_tgt_input, batch_tgt_output) in enumerate(train_loader):
                batch_src = batch_src.float().to(self.device).permute(1,0,2)
                batch_tgt_input = batch_tgt_input.float().to(self.device).permute(1,0,2)
                batch_tgt_output = batch_tgt_output.float().to(self.device).permute(1,0,2)

                optimizer.zero_grad()
                predictions = self.forward(batch_src, batch_tgt_input)
                if predictions is None: continue
                loss = criterion(predictions, batch_tgt_output)
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()
            avg_epoch_loss = epoch_loss / len(train_loader) if len(train_loader) > 0 else float('nan')
            print(f"Epoch {epoch+1}/{epochs}, Average Loss: {avg_epoch_loss:.4f}")
        print("Training finished."); self.save_model(AUTOFORMER_MODEL_PATH)

    def predict(self, series_data): # Expects 1D series_data
        if not torch_available or self.model is None: print("Cannot predict: PyTorch or model not available."); return np.array([np.nan] * self.pred_len)
        self.model.eval()
        with torch.no_grad():
            if isinstance(series_data, pd.Series): series_data = series_data.values
            if isinstance(series_data, list): series_data = np.array(series_data)
            if not isinstance(series_data, np.ndarray): print(f"Unsupported series data type for predict: {type(series_data)}"); return np.array([np.nan] * self.pred_len)
            if series_data.ndim > 1: series_data = series_data.squeeze()
            if series_data.ndim == 0: print(f"Series data is scalar, cannot process."); return np.array([np.nan]*self.pred_len)

            mean_val = series_data[-self.input_len:].mean() if len(series_data) >= self.input_len else series_data.mean()
            std_val = series_data[-self.input_len:].std() if len(series_data) >= self.input_len else series_data.std()
            std_val = std_val if std_val > 1e-8 else 1e-8

            if len(series_data) < self.input_len:
                padding_val = series_data[0] if len(series_data) > 0 else 0
                norm_series_np = (np.concatenate([np.full(self.input_len - len(series_data), padding_val), series_data]) - mean_val) / std_val
            else:
                norm_series_np = (series_data[-self.input_len:] - mean_val) / std_val

            src_tensor = torch.tensor(norm_series_np, dtype=torch.float32).unsqueeze(0).unsqueeze(-1).permute(1,0,2).to(self.device)
            current_tgt_sequence_normalized = torch.zeros((1, 1, 1), dtype=torch.float32, device=self.device)
            predictions_normalized_list = []

            for _ in range(self.pred_len):
                prediction_step_output = self.forward(src_tensor, current_tgt_sequence_normalized)
                if prediction_step_output is None: return np.array([np.nan] * self.pred_len)
                last_predicted_value_normalized = prediction_step_output[-1:, :, :]
                predictions_normalized_list.append(last_predicted_value_normalized.squeeze().cpu().item())
                current_tgt_sequence_normalized = torch.cat([current_tgt_sequence_normalized, last_predicted_value_normalized], dim=0)

            denorm_preds = (np.array(predictions_normalized_list) * std_val) + mean_val
            return denorm_preds

    def save_model(self, path):
        if not torch_available or self.model is None: return
        torch.save({
            'model_state_dict': self.model.state_dict(),
            'enc_embedding_state_dict': self.enc_embedding.state_dict(),
            'dec_embedding_state_dict': self.dec_embedding.state_dict(),
            'projection_state_dict': self.projection.state_dict(),
            'input_len': self.input_len,
            'pred_len': self.pred_len,
            'd_model': self.d_model
        }, path)
        print(f"AutoformerPredictor model saved to {path}")

    def load_model(self, path):
        if not torch_available: print("Cannot load model: PyTorch not available."); return False
        if not os.path.exists(path): print(f"Model file not found: {path}. New model needed."); return False
        try:
            checkpoint = torch.load(path, map_location=self.device)
            self.input_len = checkpoint.get('input_len', self.input_len)
            self.pred_len = checkpoint.get('pred_len', self.pred_len)
            saved_d_model = checkpoint.get('d_model', self.d_model)
            self.__init__(input_len=self.input_len, pred_len=self.pred_len, d_model=saved_d_model)
            self.model.load_state_dict(checkpoint['model_state_dict'])
            self.enc_embedding.load_state_dict(checkpoint['enc_embedding_state_dict'])
            self.dec_embedding.load_state_dict(checkpoint['dec_embedding_state_dict'])
            self.projection.load_state_dict(checkpoint['projection_state_dict'])
            self.model.eval()
            print(f"AutoformerPredictor model loaded from {path}")
            return True
        except Exception as e:
            print(f"Error loading Autoformer model: {e}. A new model might be needed.")
            return False

def create_sequences(data, input_len, pred_len): # For Autoformer training
    X, y_input, y_output = [], [], []
    for i in range(len(data) - input_len - pred_len + 1):
        X.append(data[i:(i + input_len)])
        y_dec_inp_seq = np.concatenate(([data[i + input_len -1]], data[(i + input_len):(i + input_len + pred_len -1)]))
        y_input.append(y_dec_inp_seq)
        y_output.append(data[(i + input_len):(i + input_len + pred_len)])
    return np.array(X), np.array(y_input), np.array(y_output)

def train_or_load_autoformer(df_series, input_len=60, pred_len=5, force_train=False): # For Autoformer
    if not torch_available: print("PyTorch not available for Autoformer."); return None
    autoformer_forecaster = AutoformerPredictor(input_len=input_len, pred_len=pred_len)
    if not force_train and os.path.exists(AUTOFORMER_MODEL_PATH):
        # print(f"Loading Autoformer model from {AUTOFORMER_MODEL_PATH}") # Less verbose
        if autoformer_forecaster.load_model(AUTOFORMER_MODEL_PATH):
            return autoformer_forecaster
        else:
            print(f"Failed to load Autoformer. Training new one.")
            force_train = True

    if force_train:
        print("Training new AutoformerPredictor model...")
        if df_series.isnull().any():
            print("Warning: df_series contains NaNs. Autoformer training might be affected. Consider imputation.")
            df_series = df_series.fillna(method='ffill').fillna(method='bfill').fillna(0)

        scaler = StandardScaler()
        series_scaled = scaler.fit_transform(df_series.values.reshape(-1, 1))
        X_seq, y_seq_dec_in, y_seq_out = create_sequences(series_scaled.flatten(), input_len, pred_len)

        if len(X_seq) == 0:
            print("Not enough data for Autoformer training sequences. Model will be un-trained.");
            return autoformer_forecaster

        X_seq = X_seq.reshape(X_seq.shape[0], X_seq.shape[1], 1)
        y_seq_dec_in = y_seq_dec_in.reshape(y_seq_dec_in.shape[0], y_seq_dec_in.shape[1], 1)
        y_seq_out = y_seq_out.reshape(y_seq_out.shape[0], y_seq_out.shape[1], 1)

        dataset = torch.utils.data.TensorDataset(
            torch.from_numpy(X_seq).float(),
            torch.from_numpy(y_seq_dec_in).float(),
            torch.from_numpy(y_seq_out).float()
        )
        train_loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)
        autoformer_forecaster.train_model(train_loader, epochs=20) # Consider more epochs for real training
    else:
        # print("Autoformer training skipped. Model remains un-trained if not loaded.") # Less verbose
        pass
    return autoformer_forecaster

# --- Data Fetching and Feature Engineering Functions ---
def fetch_twelve_data(symbol: str, api_key: str, start_date_str: str = None, end_date_str: str = None) -> pd.DataFrame | None:
    base_url = "https://api.twelvedata.com/time_series"
    params = {"symbol": symbol, "interval": "1day", "apikey": api_key, "format": "JSON", "outputsize": 5000}
    if start_date_str: params["start_date"] = start_date_str
    if end_date_str: params["end_date"] = end_date_str
    print(f"Fetching data for {symbol} from Twelve Data (interval=1day, from {start_date_str} to {end_date_str})...")
    try:
        response = requests.get(base_url, params=params, timeout=30)
        response.raise_for_status()
        data = response.json()
    except requests.exceptions.RequestException as e: print(f"Request failed for {symbol}: {e}"); return None
    except ValueError as e: print(f"Failed to parse JSON for {symbol}: {e}. Response: {response.text[:200]}..."); return None

    if isinstance(data, dict) and (data.get("status") == "error" or "values" not in data):
        print(f"API Error for {symbol}: {data.get('message', 'Unknown error')}"); return None
    if not isinstance(data, dict) or "values" not in data or not data["values"]:
        print(f"No data values for {symbol}, or unexpected format."); return None

    df = pd.DataFrame(data["values"]).rename(columns={'datetime': 'date'})
    for col in ['open', 'high', 'low', 'close', 'volume']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').astype('float64')
        else:
            if col in ['open', 'high', 'low', 'close']: print(f"Critical column '{col}' missing."); return None
            df[col] = 0.0
    if 'date' not in df.columns: print("Critical 'date' column missing."); return None

    df.index = pd.to_datetime(df['date'])
    df.drop(columns=['date'], inplace=True)
    df.sort_index(inplace=True)
    df.dropna(subset=[col for col in ['open', 'high', 'low', 'close'] if col in df.columns], inplace=True)
    if df.empty: print(f"No data remaining for {symbol} after initial processing."); return None
    print(f"Successfully fetched/processed {len(df)} data points for {symbol}.")
    return df

def add_technical_indicators(df: pd.DataFrame) -> pd.DataFrame:
    df_feat = df.copy()
    if not hasattr(df_feat.ta, 'rsi'): print("pandas_ta not registered. Skipping TIs."); return df_feat
    print("Adding technical indicators...")
    try:
        for col in ['open', 'high', 'low', 'close', 'volume']:
            if col in df_feat.columns: df_feat[col] = df_feat[col].astype('float64')
            else:
                if col in ['high', 'low', 'open', 'close'] and 'close' in df_feat: df_feat[col] = df_feat['close']
                elif col == 'volume': df_feat[col] = 0.0

        c, h, l, v = 'close', 'high', 'low', 'volume'
        required_cols_for_all_ta = [c,h,l,v]
        if not all(col in df_feat.columns for col in required_cols_for_all_ta):
            print("Warning: Not all OHLCV columns present, some TIs might fail or be inaccurate.")

        if c in df_feat:
            df_feat.ta.rsi(close=df_feat[c], length=14, append=True)
            df_feat.ta.rsi(close=df_feat[c], length=9, append=True, col_names='RSI_9')
            df_feat.ta.rsi(close=df_feat[c], length=25, append=True, col_names='RSI_25')
            df_feat.ta.macd(close=df_feat[c], fast=12, slow=26, signal=9, append=True)
            df_feat.ta.macd(close=df_feat[c], fast=5, slow=15, signal=9, append=True, col_names=('MACD_5_15_9', 'MACDh_5_15_9', 'MACDs_5_15_9'))
            for p in [10, 20, 50, 100, 200]:
                df_feat.ta.sma(close=df_feat[c], length=p, append=True)
                df_feat.ta.ema(close=df_feat[c], length=p, append=True)
            df_feat.ta.bbands(close=df_feat[c], length=20, std=2, append=True)
        else:
            print(f"Column '{c}' not found, skipping some TIs.")

        if all(x in df_feat.columns for x in [h,l,c]):
            df_feat.ta.atr(high=df_feat[h], low=df_feat[l], close=df_feat[c], length=14, append=True)
            df_feat.ta.adx(high=df_feat[h], low=df_feat[l], close=df_feat[c], length=14, append=True)
            df_feat.ta.stoch(high=df_feat[h], low=df_feat[l], close=df_feat[c], append=True)
            df_feat.ta.willr(high=df_feat[h], low=df_feat[l], close=df_feat[c], append=True)
            df_feat.ta.cci(high=df_feat[h], low=df_feat[l], close=df_feat[c], append=True)
        else:
            print(f"One or more of '{h}', '{l}', '{c}' not found, skipping some TIs.")

        if all(x in df_feat.columns for x in [h,l,c,v]):
            df_feat.ta.mfi(high=df_feat[h], low=df_feat[l], close=df_feat[c], volume=df_feat[v], append=True)
        else:
            print(f"One or more of '{h}', '{l}', '{c}', '{v}' not found, skipping MFI.")

        df_feat.columns = df_feat.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True)
        # print("Technical indicators added.") # Less verbose
    except Exception as e: print(f"Error adding TIs: {e}\n{traceback.format_exc()}"); return df
    return df_feat

def add_optimized_features(df: pd.DataFrame, price_col='close', volume_col='volume') -> pd.DataFrame:
    # print("Adding optimized statistical and ratio features...") # Less verbose
    df_new = df.copy()
    if price_col not in df_new.columns:
        print(f"Price column '{price_col}' not in DataFrame. Skipping optimized features."); return df_new

    df_new['returns'] = df_new[price_col].pct_change()
    safe_price = df_new[price_col].replace(0, np.nan)
    safe_price_shifted = df_new[price_col].shift(1).replace(0, np.nan)
    df_new['log_returns'] = np.log(safe_price / safe_price_shifted)

    for window in [5, 10, 20, 50]:
        df_new[f'volatility_{window}'] = df_new['log_returns'].rolling(window).std()
        df_new[f'skew_{window}'] = df_new['log_returns'].rolling(window).skew()
        df_new[f'kurtosis_{window}'] = df_new['log_returns'].rolling(window).kurt()

    if volume_col in df_new.columns and df_new[volume_col].isnull().sum() < len(df_new):
        rolling_mean_volume = df_new[volume_col].rolling(20).mean().replace(0, np.nan)
        df_new['volume_ratio'] = df_new[volume_col] / rolling_mean_volume
        df_new['price_volume'] = df_new[price_col] * df_new[volume_col]
        df_new['volume_change'] = df_new[volume_col].pct_change()

    if all(col in df_new.columns for col in ['high', 'low', 'close']):
        safe_low = df_new['low'].replace(0, np.nan)
        safe_high = df_new['high'].replace(0, np.nan)
        df_new['high_low_ratio'] = df_new['high'] / safe_low
        df_new['close_to_high_ratio'] = safe_price / safe_high
        df_new['close_to_low_ratio'] = safe_price / safe_low
        df_new['intraday_range_norm'] = (df_new['high'] - df_new['low']) / safe_price
    else:
        print("High, Low, or Close columns missing for some ratio calculations.")

    if 'RSI_14' in df_new.columns: # Ensure RSI_14 was added by add_technical_indicators
        df_new['RSI_signal'] = 0
        df_new.loc[df_new['RSI_14'] < 30, 'RSI_signal'] = 1
        df_new.loc[df_new['RSI_14'] > 70, 'RSI_signal'] = -1

    macd_col_name = 'MACD_12_26_9'
    macds_col_name = 'MACDs_12_26_9'
    if macd_col_name in df_new.columns and macds_col_name in df_new.columns:
        df_new['MACD_signal_line_cross'] = (df_new[macd_col_name] > df_new[macds_col_name]).astype(int)
    return df_new

def add_wavelet_features(df: pd.DataFrame, column='close', wavelet='mexh', scales_range=(1, 32), num_scales_to_extract=5) -> pd.DataFrame:
    df_feat = df.copy()
    # print(f"Adding CWT for '{column}' using wavelet '{wavelet}'...") # Less verbose
    if pywt is None: print("PyWavelets not available."); return df_feat
    if column not in df_feat.columns: print(f"Column '{column}' not found for wavelet. Skipping."); return df_feat

    signal = df_feat[column].values
    if len(signal) < scales_range[1] + 5: # Ensure enough data for max scale
        print(f"Signal length {len(signal)} too short for CWT with max scale {scales_range[1]}. Skipping."); return df_feat

    actual_max_scale = min(scales_range[1], len(signal) // 2 - 1) # Max scale cannot be > N/2
    if actual_max_scale < scales_range[0]:
        print(f"Max scale {actual_max_scale} too small after constraint (min_scale {scales_range[0]}). Skipping CWT."); return df_feat

    scales = np.arange(scales_range[0], actual_max_scale + 1)
    if len(scales) == 0: print("No valid scales for CWT. Skipping."); return df_feat

    try:
        coefficients, _ = pywt.cwt(signal, scales, wavelet) # coefficients shape: (num_scales, len_signal)
        coeffs_df = pd.DataFrame(coefficients.T, index=df_feat.index, columns=[f"cwt_scale_{s}" for s in scales])

        df_feat[f'{column}_cwt_mean'] = coeffs_df.mean(axis=1)
        df_feat[f'{column}_cwt_std'] = coeffs_df.std(axis=1)

        s_indices_to_extract = np.linspace(0, len(scales)-1, min(num_scales_to_extract, len(scales)), dtype=int)
        for s_idx in s_indices_to_extract:
            actual_scale_val = scales[s_idx]
            col_name_for_scale = f"cwt_scale_{actual_scale_val}" # This should match column in coeffs_df
            if col_name_for_scale in coeffs_df.columns:
                df_feat[f'{column}_cwt_energy_s{actual_scale_val}'] = coeffs_df[col_name_for_scale]**2
        # print("Wavelet features added.") # Less verbose
    except Exception as e: print(f"Error adding wavelet features: {e}\n{traceback.format_exc()}"); return df
    return df_feat

def add_entropy_features(df: pd.DataFrame, column='close', window=40) -> pd.DataFrame:
    df_feat = df.copy()
    # print(f"Adding Entropy for '{column}' (window={window})...") # Less verbose
    if ant is None: print("Antropy not available."); return df_feat
    if column not in df_feat.columns: print(f"Column '{column}' not found for entropy. Skipping."); return df_feat

    if len(df_feat) < window + 15: # Ensure enough data for rolling window calculations
        print(f"Data length {len(df_feat)} too short for entropy features with window {window}. Skipping."); return df_feat
    try:
        sig = df_feat[column].astype(float) # Ensure float for antropy functions
        df_feat[f'{column}_entropy_sample'] = sig.rolling(window=window, min_periods=window).apply(
            lambda x: ant.sample_entropy(x.dropna()) if x.dropna().shape[0] >= window//2 and x.dropna().std() > 1e-6 else np.nan, raw=False
        )
        df_feat[f'{column}_entropy_spectral'] = sig.rolling(window=window, min_periods=window).apply(
            lambda x: ant.spectral_entropy(x.dropna(), sf=1.0, method='welch',
                                           nperseg=min(x.dropna().shape[0], window // 2 if window // 2 > 0 else 1) if x.dropna().shape[0] > 1 else None
                                          ) if x.dropna().shape[0] == window and x.dropna().std() > 1e-6 else np.nan, raw=False
        )
        # print("Entropy features added.") # Less verbose
    except Exception as e: print(f"Error adding entropy features: {e}\n{traceback.format_exc()}"); return df
    return df_feat

def add_advanced_technical_features(df: pd.DataFrame, price_col='close', high_col='high', low_col='low', volume_col='volume') -> pd.DataFrame:
    # print("Adding advanced TIs...") # Less verbose
    df_new = df.copy()
    if not hasattr(df_new.ta, 'mom'): print("pandas_ta not registered. Skipping advanced TIs."); return df_new
    try:
        if not all(c in df_new.columns for c in [price_col, high_col, low_col]):
            print(f"Missing one or more of {price_col}, {high_col}, {low_col} for adv TIs. Skipping."); return df_new
        for col in [price_col, high_col, low_col]: df_new[col] = df_new[col].astype(float)
        if volume_col in df_new.columns: df_new[volume_col] = df_new[volume_col].astype(float)

        df_new.ta.mom(close=df_new[price_col], append=True)
        df_new.ta.roc(close=df_new[price_col], append=True)
        df_new.ta.natr(high=df_new[high_col], low=df_new[low_col], close=df_new[price_col], append=True)
        df_new.ta.aroon(high=df_new[high_col], low=df_new[low_col], append=True)
        df_new.ta.stc(close=df_new[price_col], tclength=23, fast=50, slow=100, factor=0.5, append=True, col_names=('STC_23_50_05', 'STCD_23_50_05', 'STCK_23_50_05'))

        if volume_col in df_new.columns and df_new[volume_col].isnull().sum() < len(df_new):
            df_new.ta.pvol(close=df_new[price_col], volume=df_new[volume_col], append=True)
            df_new.ta.cmf(high=df_new[high_col], low=df_new[low_col], close=df_new[price_col], volume=df_new[volume_col], append=True)

        df_new.columns = df_new.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True) # Final sanitize
        # print("Advanced TIs added.") # Less verbose
    except Exception as e: print(f"Error adding advanced TIs: {e}\n{traceback.format_exc()}"); return df
    return df_new

def add_transformer_features_conceptual(df: pd.DataFrame, column='close', sequence_length=20) -> pd.DataFrame:
    df_feat = df.copy()
    # print(f"Adding Transformer-inspired conceptual features for '{column}'...") # Less verbose
    if column not in df_feat.columns: print(f"Column '{column}' not found for conceptual Transformer features. Skipping."); return df_feat
    if len(df_feat) < sequence_length + 5: print(f"Data too short for conceptual Transformer features. Skipping."); return df_feat

    feature_col_base = f"{column}_trans_seq"
    for col_suffix in ['mean', 'std', 'trend', 'volatility', 'autocorr1']:
        df_feat[f'{feature_col_base}_{col_suffix}'] = np.nan
    try:
        data_series = df_feat[column].values
        windows = np.lib.stride_tricks.sliding_window_view(data_series, sequence_length)
        results = {key: [np.nan] * (sequence_length -1) for key in ['mean', 'std', 'trend', 'volatility', 'autocorr1']}

        for seq in windows:
            if np.isnan(seq).any(): # Skip windows with NaNs
                for key in results: results[key].append(np.nan)
                continue
            mean_val, std_val = np.mean(seq), np.std(seq)
            norm_seq = (seq - mean_val) / std_val if std_val > 1e-8 else np.zeros_like(seq)

            results['mean'].append(np.mean(norm_seq))
            results['std'].append(np.std(norm_seq))
            current_trend, current_vol, current_ac = 0.0, 0.0, 0.0 # Defaults for safety

            if len(norm_seq) > 1:
                try:
                    fit_params = np.polyfit(np.arange(len(norm_seq)), norm_seq, 1)
                    current_trend = fit_params[0] if not np.isnan(fit_params[0]) else 0.0
                except (np.linalg.LinAlgError, ValueError): pass # Keep 0.0

                diff_norm_seq = np.diff(norm_seq)
                current_vol = np.std(diff_norm_seq) if len(diff_norm_seq) > 0 else 0.0

                if len(norm_seq) >= 2: # Autocorrelation needs at least 2 points
                    s1, s2 = norm_seq[:-1], norm_seq[1:]
                    if len(s1) >= 1 and np.std(s1) > 1e-8 and np.std(s2) > 1e-8:
                        try:
                            corr_matrix = np.corrcoef(s1, s2)
                            current_ac = corr_matrix[0, 1] if not np.isnan(corr_matrix[0, 1]) else 0.0
                        except (ValueError, IndexError): pass # Keep 0.0
            results['trend'].append(current_trend)
            results['volatility'].append(current_vol)
            results['autocorr1'].append(current_ac)

        for key, values in results.items():
            if len(values) == len(df_feat):
                df_feat[f'{feature_col_base}_{key}'] = values
            else:
                # print(f"Warning: Length mismatch for conceptual feature {key}. Expected {len(df_feat)}, got {len(values)}. Padding with NaN.") # Less verbose
                padded_values = np.full(len(df_feat), np.nan)
                if len(values) > 0 : padded_values[-len(values):] = values # Align at the end
                df_feat[f'{feature_col_base}_{key}'] = padded_values
        # print("Conceptual Transformer-inspired features added.") # Less verbose
    except Exception as e: print(f"Error in conceptual Transformer features: {e}\n{traceback.format_exc()}"); return df
    return df_feat

def detect_regimes_simple(df: pd.DataFrame, column='close', window=20) -> pd.DataFrame:
    df_reg = df.copy()
    # print(f"Detecting regimes (simplified volatility-based) for {column}...") # Less verbose
    if column not in df_reg.columns:
        print(f"'{column}' not found. Skipping simple regimes.")
        df_reg['regime_simple'] = 0
        return df_reg

    if 'log_returns' not in df_reg.columns:
        safe_price = df_reg[column].replace(0, np.nan)
        safe_price_shifted = df_reg[column].shift(1).replace(0, np.nan)
        df_reg['log_returns_temp_for_regime'] = np.log(safe_price / safe_price_shifted)
        returns_col_for_regime = 'log_returns_temp_for_regime'
    else:
        returns_col_for_regime = 'log_returns'

    returns = df_reg[returns_col_for_regime].dropna()
    if returns.empty:
        print("No valid returns for simple regime detection. Skipping.")
        df_reg['regime_simple'] = 0
        if 'log_returns_temp_for_regime' in df_reg.columns: df_reg.drop(columns=['log_returns_temp_for_regime'], inplace=True)
        return df_reg

    rolling_vol = returns.rolling(window=window, min_periods=window//2 if window//2 > 0 else 1).std()
    df_reg['regime_simple'] = 0 # Default: medium volatility (class 0)

    if not rolling_vol.dropna().empty:
        vol_low_thresh = rolling_vol.quantile(0.33)
        vol_high_thresh = rolling_vol.quantile(0.67)
        df_reg.loc[rolling_vol.index[rolling_vol <= vol_low_thresh], 'regime_simple'] = 1  # Low vol (class 1)
        df_reg.loc[rolling_vol.index[rolling_vol > vol_high_thresh], 'regime_simple'] = 2   # High vol (class 2)
    else:
        print("Not enough data for rolling volatility percentile calculation. Defaulting simple regimes to 0.")

    df_reg['regime_simple'] = df_reg['regime_simple'].fillna(method='bfill').fillna(method='ffill').fillna(0)
    if 'log_returns_temp_for_regime' in df_reg.columns: df_reg.drop(columns=['log_returns_temp_for_regime'], inplace=True)

    print(f"Simple Regimes (0:Med,1:Low,2:High):\n{df_reg['regime_simple'].value_counts(normalize=True, dropna=False).sort_index()*100} %")
    return df_reg

def balanced_target_definition(df: pd.DataFrame, column='close', periods=5, lower_q_thresh=0.45, upper_q_thresh=0.55) -> pd.DataFrame:
    df_t = df.copy()
    # print(f"Balanced target definition for '{column}' over {periods} periods...") # Less verbose
    if column not in df_t.columns:
        print(f"'{column}' not found for target. Defaulting target.")
        df_t['target'] = 0
        return df_t

    df_t[column] = pd.to_numeric(df_t[column], errors='coerce').replace(0, np.nan)
    df_t['future_log_return_target'] = np.log(df_t[column].shift(-periods) / df_t[column])
    valid_returns = df_t['future_log_return_target'].dropna()
    df_t['target'] = 0 # Default class (e.g., Hold/Neutral)

    if len(valid_returns) > 20: # Need enough data for meaningful quantiles
        lower_q_val = valid_returns.quantile(lower_q_thresh)
        upper_q_val = valid_returns.quantile(upper_q_thresh)
        if lower_q_val >= upper_q_val and upper_q_val > 0 : lower_q_val = upper_q_val * 0.99 # small adjustment
        elif lower_q_val >= upper_q_val and upper_q_val < 0 : upper_q_val = lower_q_val * 0.99
        df_t.loc[df_t['future_log_return_target'] < lower_q_val, 'target'] = 0
        df_t.loc[df_t['future_log_return_target'] > upper_q_val, 'target'] = 1
    else:
        print("Not enough valid returns for quantile-based target balancing. Default target (all 0s) used or target may be skewed.")

    df_t.drop(columns=['future_log_return_target'], inplace=True, errors='ignore')
    print(f"Target distribution:\n{df_t['target'].value_counts(normalize=True, dropna=False)*100}")
    return df_t

def discover_causal_structure(df_features: pd.DataFrame, target_col='target', price_c='close', max_feats=10, symbol="") -> CausalModel | None:
    print(f"\nDiscovering causal structure for {symbol} using DoWhy...")
    if not dowhy_available or CausalModel is None:
        print("DoWhy not available.")
        return None

    df_c = df_features.copy()
    if target_col not in df_c.columns or df_c[target_col].isnull().all():
        print(f"Target '{target_col}' missing for causal discovery.")
        return None
    df_c[target_col] = pd.to_numeric(df_c[target_col], errors='coerce')
    cand_cols = [c for c in df_c.columns if pd.api.types.is_numeric_dtype(df_c[c]) and c != target_col and df_c[c].notnull().any() and df_c[c].var() > 1e-6]
    if not cand_cols:
        print("No numeric candidate columns with variance for causal discovery.")
        return None

    df_subset_for_causal = df_c[cand_cols + [target_col]].copy()
    df_subset_for_causal.replace([np.inf, -np.inf], np.nan, inplace=True)
    if cand_cols:
        scaler_causal = StandardScaler()
        df_subset_for_causal[cand_cols] = scaler_causal.fit_transform(df_subset_for_causal[cand_cols])
    df_subset_for_causal.dropna(inplace=True)
    if df_subset_for_causal.empty or target_col not in df_subset_for_causal.columns or df_subset_for_causal[target_col].nunique() < 1:
        print("Not enough data post-cleaning/scaling for causal discovery.")
        return None

    cwt_mean_col = f"{price_c}_cwt_mean" if f"{price_c}_cwt_mean" in df_subset_for_causal.columns else 'close_cwt_mean'
    cwt_std_col = f"{price_c}_cwt_std" if f"{price_c}_cwt_std" in df_subset_for_causal.columns else 'close_cwt_std'
    entropy_sample_col = f"{price_c}_entropy_sample" if f"{price_c}_entropy_sample" in df_subset_for_causal.columns else 'close_entropy_sample'
    potential_causes = ['RSI_14', 'MACDh_12_26_9', 'ADX_14', 'ATR_14', cwt_mean_col, cwt_std_col, entropy_sample_col, 'regime_simple', 'volatility_20', 'log_returns', 'BBP_2020', 'BBB_2020']
    graph_feats = [c for c in potential_causes if c in df_subset_for_causal.columns and c != target_col and df_subset_for_causal[c].nunique() > 1]
    if not graph_feats:
        print("Predefined causal graph_feats not suitable or not found, selecting top varying features (after scaling).")
        num_to_select = min(max_feats, len(cand_cols))
        if num_to_select > 0:
            graph_feats = df_subset_for_causal[cand_cols].var().nlargest(num_to_select).index.tolist()
        else:
            print("No candidate columns for graph_feats fallback.")
            return None
    if not graph_feats:
        print("No suitable graph features for causal discovery.")
        return None

    final_df_for_causal_model = df_subset_for_causal[graph_feats + [target_col]].copy()
    if final_df_for_causal_model.empty or final_df_for_causal_model.shape[0] < 20 or final_df_for_causal_model[target_col].nunique() < 1:
        print("Final DF for causal model too small or target has no variation.")
        return None
    print(f"DoWhy using graph features: {graph_feats} for Outcome: {target_col}")
    treatment_var = graph_feats[0]
    graph_str = "digraph { " + "; ".join([f'"{f}" -> "{target_col}"' for f in graph_feats]) + " }"
    print(f"Generated Causal Graph:\n{graph_str}")
    try:
        model = CausalModel(data=final_df_for_causal_model, treatment=treatment_var, outcome=target_col, graph=graph_str)
        print("DoWhy CausalModel created.")
        return model
    except Exception as e:
        print(f"DoWhy CausalModel error: {e}\n{traceback.format_exc()}")
        return None

def prepare_ml_data(df: pd.DataFrame, target_col='target', test_split_size=0.15, min_test_samples=50):
    # print("Preparing ML data...") # Less verbose
    if target_col not in df.columns:
        print(f"Target '{target_col}' missing.")
        return None, None, None, None, None

    cols_to_drop_base = ['open', 'high', 'low', 'close', 'volume', 'returns']
    cols_to_drop_dynamic = [c for c in df.columns if 'target_' in c and c != target_col] + \
                           [c for c in df.columns if 'future_return' in c]

    all_cols_to_drop = list(set(cols_to_drop_base + cols_to_drop_dynamic))
    if target_col in all_cols_to_drop:
        all_cols_to_drop.remove(target_col)

    X = df.drop(columns=[c for c in all_cols_to_drop if c in df.columns] + [target_col], errors='ignore')
    y = df[target_col].copy()

    if y.isnull().all():
        print("Target is all NaN.")
        return None, None, None, None, None

    valid_target_mask = y.notna()
    X = X.loc[valid_target_mask]
    y = y.loc[valid_target_mask]

    if X.empty or y.empty:
        print("X or y empty after target NaN filter.")
        return None, None, None, None, None

    X.replace([np.inf, -np.inf], np.nan, inplace=True)

    if X.isnull().any().any():
        # print(f"NaNs in X before median imputation: {X.isnull().sum().sum()}") # Less verbose
        for col in X.columns:
            if X[col].isnull().any():
                if pd.api.types.is_numeric_dtype(X[col]):
                    X[col] = X[col].fillna(X[col].median())
                else:
                    X[col] = X[col].fillna(X[col].mode()[0] if not X[col].mode().empty else "Unknown")

    if X.isnull().any().any():
        print(f"Warning: NaNs still present after imputation. Dropping rows with NaNs in X. Nulls per col:\n{X.isnull().sum()[X.isnull().sum()>0]}")
        X.dropna(axis=0, how='any', inplace=True)
        y = y.loc[X.index]

    if X.empty or y.empty:
        print("X or y empty after internal NaN handling.")
        return None, None, None, None, None
    # print(f"Data shape post-NaN handling in prepare_ml_data: X={X.shape}, y={y.shape}") # Less verbose

    if len(X) < min_test_samples * 2:
        print(f"Not enough data ({len(X)} rows) for robust train/test split. Min required for split: {min_test_samples*2}.")
        return None, None, None, None, None

    n_samples = len(X)
    test_size_abs = max(min_test_samples, int(n_samples * test_split_size))

    if n_samples - test_size_abs < min_test_samples:
        test_size_abs = n_samples - min_test_samples

    if test_size_abs < 1 and n_samples > 0:
        test_size_abs = 1
    elif test_size_abs < 1:
        print(f"Cannot make meaningful split (test_size_abs < 1).")
        return None, None, None, None, None

    train_size = n_samples - test_size_abs
    if train_size < 1:
        print(f"Train size too small ({train_size}). Cannot split.")
        return None, None, None, None, None

    X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]
    y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]

    if X_train.empty or y_train.empty or X_test.empty or y_test.empty:
        print("Train/Test set empty post-split.")
        return None, None, None, None, None
    print(f"Train shapes: X_train={X_train.shape}, y_train={y_train.shape}; Test shapes: X_test={X_test.shape}, y_test={y_test.shape}")

    numeric_cols_xtrain = X_train.select_dtypes(include=np.number).columns
    scaler = None

    if not numeric_cols_xtrain.empty:
        scaler = StandardScaler()
        X_train_scaled_np = scaler.fit_transform(X_train[numeric_cols_xtrain])
        X_train_scaled_df = pd.DataFrame(X_train_scaled_np, columns=numeric_cols_xtrain, index=X_train.index)

        X_train_final = X_train.copy()
        X_train_final[numeric_cols_xtrain] = X_train_scaled_df

        numeric_cols_xtest = X_test.select_dtypes(include=np.number).columns
        common_numeric_cols = [col for col in numeric_cols_xtrain if col in numeric_cols_xtest]

        X_test_final = X_test.copy()
        if common_numeric_cols:
            X_test_scaled_np = scaler.transform(X_test[common_numeric_cols])
            X_test_scaled_df = pd.DataFrame(X_test_scaled_np, columns=common_numeric_cols, index=X_test[common_numeric_cols].index)
            X_test_final[common_numeric_cols] = X_test_scaled_df
        else:
            print("No common numeric columns to scale in X_test, or X_test has no numeric columns that were scaled in train.")

        return X_train_final, X_test_final, y_train, y_test, scaler
    else:
        print("No numeric columns in X_train for scaling.")
        return X_train, X_test, y_train, y_test, None

def lgbm_objective(trial, X_train, y_train, X_val, y_val, base_params):
    params = {
        **base_params,
        'boosting_type': 'gbdt',
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=50),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 10, 50),
        'max_depth': trial.suggest_int('max_depth', -1, 10),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 0, 7),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 1.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 1.0, log=True),
        'min_gain_to_split': trial.suggest_float('min_gain_to_split', 0.0, 0.1) # Added for Optuna
    }
    if params.get('num_class') is None and 'num_class' in params: # Should not be needed if base_params is set right
        del params['num_class']

    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              eval_metric=base_params.get('metric', 'logloss'), # Ensure metric is passed from base
              callbacks=[lgb.early_stopping(30, verbose=False)])
    y_proba_val = model.predict_proba(X_val)
    return log_loss(y_val, y_proba_val)

def optimize_lgbm_hyperparameters(X_train: pd.DataFrame, y_train: pd.Series, base_params: dict, n_trials=30, validation_ratio=0.2) -> dict:
    if not optuna_available:
        print("Optuna not available. Using default HPs.")
        return optimized_lightgbm_params() # Return default set

    print(f"Optimizing LightGBM HPs with Optuna ({n_trials} trials)...")
    if len(X_train) * validation_ratio < 1 or len(X_train) * (1-validation_ratio) < 1:
        print("Too few samples for Optuna validation split. Using default HPs.")
        return optimized_lightgbm_params()

    n_val_samples = int(len(X_train) * validation_ratio)
    if n_val_samples == 0 and len(X_train) > 1: n_val_samples = 1
    elif n_val_samples == 0 :
        print("Cannot create validation set for Optuna (0 samples). Using default HPs.")
        return optimized_lightgbm_params()

    n_train_opt_samples = len(X_train) - n_val_samples
    if n_train_opt_samples == 0:
        print("Train set for Optuna is empty after split. Using default HPs.")
        return optimized_lightgbm_params()

    X_train_opt, X_val_opt = X_train.iloc[:n_train_opt_samples], X_train.iloc[n_train_opt_samples:]
    y_train_opt, y_val_opt = y_train.iloc[:n_train_opt_samples], y_train.iloc[n_train_opt_samples:]

    # Ensure base_params for objective has 'num_class' if multiclass
    # This should already be handled when base_params is created before calling this function
    study = optuna.create_study(direction='minimize') # Minimize log_loss
    study.optimize(lambda trial: lgbm_objective(trial, X_train_opt, y_train_opt, X_val_opt, y_val_opt, base_params),
                   n_trials=n_trials, show_progress_bar=True)

    print(f"Best Optuna trial for LightGBM: Value={study.best_value:.4f}, Params={study.best_params}")
    # Return only the tuned parameters, base_params will be merged by the caller
    return study.best_params

def train_lightgbm_model(X_train, y_train, X_test, y_test, optimized_params=None):
    print("Training LightGBM model...")
    if X_train is None or X_train.empty or y_train is None or y_train.empty:
        print("X_train or y_train is empty. Skipping LightGBM training.")
        return None, None

    y_train_squeezed = y_train.squeeze()
    y_test_squeezed = y_test.squeeze() if y_test is not None else pd.Series()
    unique_labels_train = sorted(y_train_squeezed.unique())
    num_classes = len(unique_labels_train)

    if num_classes <= 1:
        print(f"Only {num_classes} class(es) in y_train. Skipping LightGBM training.")
        return None, None

    # Define base parameters for LightGBM
    current_params = optimized_lightgbm_params() # Start with defaults
    current_params['objective'] = 'multiclass' if num_classes > 2 else 'binary'
    current_params['metric'] = 'multi_logloss' if num_classes > 2 else 'binary_logloss'
    if num_classes > 2:
        current_params['num_class'] = num_classes
    else: # Ensure num_class is not present for binary
        if 'num_class' in current_params:
            del current_params['num_class']


    if optimized_params and isinstance(optimized_params, dict):
        print("Using Optuna-optimized parameters.")
        current_params.update(optimized_params) # Merge Optuna's best params
    else:
        print("Using default (or non-Optuna optimized) LightGBM parameters.")


    label_map = {label: i for i, label in enumerate(unique_labels_train)}
    y_train_mapped = y_train_squeezed.map(label_map)
    model = lgb.LGBMClassifier(**current_params)
    eval_set_data = None
    valid_eval_indices = None
    y_test_mapped_for_eval = None

    if X_test is not None and not X_test.empty and not y_test_squeezed.empty:
        y_test_mapped = y_test_squeezed.map(label_map).fillna(-1).astype(int)
        valid_eval_indices = (y_test_mapped != -1)
        if valid_eval_indices.any():
            y_test_mapped_for_eval = y_test_mapped[valid_eval_indices]
            # Ensure X_test for eval_set has same columns as X_train
            X_test_eval = X_test[valid_eval_indices][X_train.columns] if all(c in X_test.columns for c in X_train.columns) else X_test[valid_eval_indices]

            eval_set_data = (X_test_eval, y_test_mapped_for_eval)

    if eval_set_data:
        model.fit(X_train, y_train_mapped, eval_set=[eval_set_data],
                  eval_metric=current_params.get('metric'), # Use metric from current_params
                  callbacks=[lgb.early_stopping(30, verbose=False)])
    else:
        print("Warning: No valid eval set. Fitting on full training data without early stopping based on eval set.")
        model.fit(X_train, y_train_mapped)

    feat_imp_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': model.feature_importances_}).sort_values(by='Importance', ascending=False)
    print("\nTop 10 features:\n", feat_imp_df.head(10))

    if eval_set_data and y_test_mapped_for_eval is not None and not y_test_mapped_for_eval.empty:
        # Ensure X_test for predict has same columns as X_train
        X_test_predict = X_test[valid_eval_indices][X_train.columns] if all(c in X_test.columns for c in X_train.columns) else X_test[valid_eval_indices]
        y_pred_mapped_on_valid = model.predict(X_test_predict)
        y_proba_on_valid = model.predict_proba(X_test_predict)
        acc = accuracy_score(y_test_mapped_for_eval, y_pred_mapped_on_valid)
        print(f"\n🎯 Accuracy on mapped test data: {acc:.4f}")

        if current_params['objective'] == 'binary' and y_proba_on_valid.shape[1] == 2:
            try:
                auc = roc_auc_score(y_test_mapped_for_eval, y_proba_on_valid[:, 1])
                print(f"📊 AUC: {auc:.4f}")
            except ValueError as e_auc:
                print(f"AUC Calculation Error: {e_auc}")
        print("\nClassification Report (on mapped and valid test labels):")
        try:
            report_labels = sorted(np.unique(np.concatenate((y_test_mapped_for_eval.unique(), pd.Series(y_pred_mapped_on_valid).unique()))))
            print(classification_report(y_test_mapped_for_eval, y_pred_mapped_on_valid, labels=report_labels, zero_division=0))
        except Exception as e_cr:
            print(f"Classification Report Error: {e_cr}")
    else:
        print("No valid test samples for evaluation after mapping, or X_test/y_test was not provided.")
    return model, feat_imp_df

def plot_feature_importance(feature_importance_df, top_n=20, symbol_for_plot="", min_bar_height=0.05):
    if feature_importance_df is None or feature_importance_df.empty:
        print("No feature importance to plot.")
        return

    plot_data = feature_importance_df.head(top_n).copy()
    if plot_data.empty:
        print("No features in plot_data after head(top_n).")
        return

    max_importance = plot_data['Importance'].max()
    min_threshold = max(max_importance * 0.02, 1e-6) # Use a small absolute minimum if relative is too small
    plot_data['Plot_Importance'] = np.maximum(plot_data['Importance'], min_threshold)

    plt.figure(figsize=(14, max(8, min(top_n, len(plot_data)) * 0.5)))
    ax = sns.barplot(x='Plot_Importance', y='Feature', data=plot_data, palette="viridis", orient='h')
    for i, row_data in enumerate(plot_data.itertuples()): # Use itertuples for easier access
        original_val = row_data.Importance # Get original importance for label
        plot_val = row_data.Plot_Importance # Get the value used for plotting the bar
        ax.text(plot_val + max_importance * 0.01, i, f'{original_val:.0f}', va='center', fontsize=9, fontweight='bold') # Changed to .0f for integer display

    plt.title(f'Top {top_n} Feature Importances for {symbol_for_plot} (LightGBM)', fontsize=16, fontweight='bold', pad=20)
    plt.xlabel('Importance', fontsize=12, fontweight='bold')
    plt.ylabel('Feature', fontsize=12, fontweight='bold')
    plt.grid(axis='x', alpha=0.3)
    plt.tight_layout()
    # plt.show() # Comment out for non-interactive environments if needed
    # Instead of plt.show(), save the figure
    plot_filename = f"feature_importance_{symbol_for_plot}.png"
    plt.savefig(plot_filename)
    print(f"Feature importance plot saved to {plot_filename}")
    plt.close() # Close the plot to free up memory


def export_lgbm_to_onnx(lgbm_model, X_sample_df, file_path="lgbm_model.onnx", target_opset=12):
    print(f"\nExporting LGBM model to ONNX: {file_path} (opset={target_opset})")
    if not all([onnx_available, skl2onnx_available, onnxmltools_available, (FloatTensorType is not None)]):
        print("One or more ONNX libraries missing or FloatTensorType not imported. Skipping ONNX export.")
        return None
    if lgbm_model is None or X_sample_df is None or X_sample_df.empty:
        print("Model or sample data empty for ONNX. Skipping.")
        return None
    try:
        initial_type = [('float_input', FloatTensorType([None, X_sample_df.shape[1]]))]
        converted_model = onnxmltools.convert_lightgbm(lgbm_model, initial_types=initial_type, target_opset=target_opset)
        with open(file_path, "wb") as f:
            f.write(converted_model.SerializeToString())
        print(f"Model exported to ONNX: {file_path}")
        onnx.checker.check_model(file_path)
        print("ONNX model check OK.")
        return file_path
    except Exception as e:
        print(f"Error exporting LGBM to ONNX: {e}. Fallback to pickle.")
        try:
            import pickle
            pkl_path = file_path.replace('.onnx', '.pkl')
            with open(pkl_path, 'wb') as pf:
                pickle.dump(lgbm_model, pf)
            print(f"Model saved as pickle: {pkl_path}")
            return pkl_path
        except Exception as ep:
            print(f"Pickle save error: {ep}")
            return None

def simple_feature_selection_fallback(X_train, y_train, max_features=20):
    print("Using simple variance-based feature selection fallback...")
    if X_train.empty: return pd.DataFrame()
    X_train_numeric = X_train.select_dtypes(include=np.number)
    if X_train_numeric.empty:
        print("No numeric features for variance selection. Returning first few columns if available.")
        return pd.DataFrame({'Feature': X_train.columns[:max_features].tolist()})
    variance_scores = X_train_numeric.var().sort_values(ascending=False)
    num_features_to_select = min(max_features, len(variance_scores))
    selected_features = variance_scores.head(num_features_to_select).index.tolist()
    return pd.DataFrame({'Feature': selected_features, 'Score': variance_scores.head(num_features_to_select).values})

def prioritized_feature_selection(X_train, y_train, causal_ranking, max_features=25):
    print("Prioritized feature selection: Causal Ranking + Mutual Information...")
    if X_train.empty or y_train.empty:
        print("X_train or y_train is empty in prioritized_feature_selection. Returning empty DataFrame.")
        return pd.DataFrame()

    top_causal_features = []
    num_causal_to_select = 0
    if causal_ranking and isinstance(causal_ranking, list) and all(isinstance(item, tuple) and len(item)==2 for item in causal_ranking):
        num_causal_to_select = min(len(causal_ranking), max_features // 2)
        if num_causal_to_select > 0:
            print(f"Selecting up to {num_causal_to_select} features from causal ranking.")
            for feat, score in causal_ranking[:num_causal_to_select]:
                if feat in X_train.columns:
                    top_causal_features.append(feat)
                else:
                    print(f"Causal feature '{feat}' not in X_train.columns. Skipping.")
    else:
        print("No valid causal ranking provided or num_causal_to_select is 0.")

    # print(f"Features selected from causal ranking: {top_causal_features}") # Less verbose
    remaining_slots = max_features - len(top_causal_features)
    stat_selected_features = []

    if remaining_slots > 0:
        features_for_stat_selection = [f for f in X_train.columns if f not in top_causal_features]
        if features_for_stat_selection:
            X_remaining_for_stat = X_train[features_for_stat_selection]
            y_train_squeezed = y_train.squeeze()

            if y_train_squeezed.nunique() > 1 and not X_remaining_for_stat.empty:
                X_remaining_numeric = X_remaining_for_stat.select_dtypes(include=np.number)
                if not X_remaining_numeric.empty:
                    num_stat_to_select = min(remaining_slots, X_remaining_numeric.shape[1])
                    if num_stat_to_select > 0:
                        try:
                            # print(f"Selecting up to {num_stat_to_select} features using Mutual Information from {X_remaining_numeric.shape[1]} numeric features.") # Less verbose
                            selector_mi = SelectKBest(mutual_info_classif, k=num_stat_to_select)
                            selector_mi.fit(X_remaining_numeric, y_train_squeezed)
                            stat_selected_features = X_remaining_numeric.columns[selector_mi.get_support()].tolist()
                            # print(f"Features selected from Mutual Information: {stat_selected_features}") # Less verbose
                        except Exception as e_mi:
                            print(f"Error in MI based feature selection: {e_mi}. Proceeding without these stat features.")
                    # else: print("No statistical features to select (num_stat_to_select is 0).") # Less verbose
                # else: print("No numeric features remaining for MI based statistical selection.") # Less verbose
            # else: print("Not enough target variance or no remaining features for MI based selection.") # Less verbose
        # else: print("No features remaining for MI based statistical selection (features_for_stat_selection is empty).") # Less verbose
    # else: print("No remaining slots for statistical feature selection.") # Less verbose

    final_selected_features = list(dict.fromkeys(top_causal_features + stat_selected_features))
    if not final_selected_features and not X_train.empty:
        print("No features from prioritized selection, falling back to simple variance-based selection.")
        simple_fallback_df = simple_feature_selection_fallback(X_train, y_train, max_features)
        if simple_fallback_df is not None and 'Feature' in simple_fallback_df.columns:
            final_selected_features = simple_fallback_df['Feature'].tolist()
        else:
            final_selected_features = X_train.columns[:max_features].tolist()

    # print(f"Total prioritized features selected: {len(final_selected_features)}. Top 5: {final_selected_features[:5] if final_selected_features else 'None'}") # Less verbose
    return pd.DataFrame({'Feature': final_selected_features})

def configure_extended_context(base_context=512, data_length=750):
    max_possible_context = int(data_length * 0.7)
    extended_contexts = {
        'short_term': min(256, max_possible_context, data_length - 60),
        'medium_term': min(512, max_possible_context, data_length - 60),
        'long_term': min(1024, max_possible_context, data_length - 60),
        'adaptive': min(base_context * 2, max_possible_context, data_length - 60)
    }
    extended_contexts['adaptive'] = max(extended_contexts['adaptive'], 64)
    # print(f"Context window options for {data_length} data points (ensuring >60 points remain):") # Less verbose
    # for name, length in extended_contexts.items(): # Less verbose
        # print(f"  {name}: {max(1, length)} days") # Less verbose
    return extended_contexts

def add_multitimeframe_features(df, price_col='close'):
    df_mtf = df.copy()
    if price_col not in df_mtf.columns:
        print(f"Price column '{price_col}' not in DataFrame. Skipping multi-timeframe features.")
        return df_mtf

    df_mtf[f'{price_col}_weekly_mean'] = df_mtf[price_col].rolling(5, min_periods=1).mean()
    df_mtf[f'{price_col}_weekly_std'] = df_mtf[price_col].rolling(5, min_periods=1).std()
    df_mtf[f'{price_col}_weekly_max'] = df_mtf[price_col].rolling(5, min_periods=1).max()
    df_mtf[f'{price_col}_weekly_min'] = df_mtf[price_col].rolling(5, min_periods=1).min()

    df_mtf[f'{price_col}_monthly_mean'] = df_mtf[price_col].rolling(21, min_periods=1).mean()
    df_mtf[f'{price_col}_monthly_std'] = df_mtf[price_col].rolling(21, min_periods=1).std()
    df_mtf[f'{price_col}_monthly_trend'] = df_mtf[price_col].rolling(21, min_periods=2).apply(
        lambda x: np.polyfit(range(len(x)), x, 1)[0] if len(x) >= 2 else np.nan, raw=False
    )
    df_mtf[f'{price_col}_quarterly_mean'] = df_mtf[price_col].rolling(63, min_periods=1).mean()
    df_mtf[f'{price_col}_quarterly_volatility'] = df_mtf[price_col].rolling(63, min_periods=1).std() / df_mtf[f'{price_col}_quarterly_mean'].replace(0, np.nan)

    df_mtf[f'{price_col}_weekly_monthly_ratio'] = df_mtf[f'{price_col}_weekly_mean'] / df_mtf[f'{price_col}_monthly_mean'].replace(0, np.nan)
    df_mtf[f'{price_col}_monthly_quarterly_ratio'] = df_mtf[f'{price_col}_monthly_mean'] / df_mtf[f'{price_col}_quarterly_mean'].replace(0, np.nan)

    # print(f"Added {11} multi-timeframe features") # Less verbose
    return df_mtf

def detect_volatility_regimes(returns, window=21, threshold_multiplier=1.5):
    if returns.empty or len(returns) < window:
        # print(f"Not enough return data ({len(returns)}) for volatility regime detection with window {window}. Defaulting.") # Less verbose
        return pd.Series(1, index=returns.index)

    rolling_vol = returns.rolling(window, min_periods=window // 2 if window // 2 > 0 else 1).std()
    if rolling_vol.dropna().empty:
        # print("Rolling volatility is all NaN. Defaulting regimes.") # Less verbose
        return pd.Series(1, index=returns.index)

    vol_median = rolling_vol.median()
    if pd.isna(vol_median) or vol_median == 0:
        vol_median = rolling_vol.mean()
        if pd.isna(vol_median) or vol_median == 0:
            # print("Median and Mean of rolling volatility are NaN/zero. Cannot determine thresholds. Defaulting regimes.") # Less verbose
            return pd.Series(1, index=returns.index)

    high_vol_threshold = vol_median * threshold_multiplier
    low_vol_threshold = vol_median / threshold_multiplier
    regimes = pd.Series(1, index=returns.index, dtype=int)
    regimes[rolling_vol >= high_vol_threshold] = 2
    regimes[rolling_vol <= low_vol_threshold] = 0
    regimes = regimes.fillna(method='ffill').fillna(1)
    # regime_counts = regimes.value_counts().sort_index() # Less verbose
    # print(f"Volatility regime distribution:") # Less verbose
    # print(f"  Low volatility (0): {regime_counts.get(0, 0)} periods") # Less verbose
    # print(f"  Normal volatility (1): {regime_counts.get(1, 0)} periods") # Less verbose
    # print(f"  High volatility (2): {regime_counts.get(2, 0)} periods") # Less verbose
    return regimes

def add_regime_features(df, returns_col='log_returns', price_col='close'):
    df_rf = df.copy()
    if returns_col not in df_rf.columns:
        print(f"Returns column '{returns_col}' not found. Skipping regime features.")
        return df_rf
    if price_col not in df_rf.columns:
        print(f"Price column '{price_col}' not found for regime-adjusted MAs. Skipping those.")

    regimes = detect_volatility_regimes(df_rf[returns_col])
    df_rf['volatility_regime'] = regimes
    df_rf['regime_0'] = (regimes == 0).astype(int)
    df_rf['regime_1'] = (regimes == 1).astype(int)
    df_rf['regime_2'] = (regimes == 2).astype(int)
    num_regime_features_added = 4
    if price_col in df_rf.columns:
        for window in [10, 20, 50]:
            df_rf[f'sma_{window}_regime_adj'] = df_rf[price_col].rolling(window, min_periods=1).mean() * (1 + 0.1 * regimes)
            num_regime_features_added +=1
    # print(f"Added {num_regime_features_added} regime-based features") # Less verbose
    return df_rf

def optimized_lightgbm_params():
    # These are reasonable defaults if Optuna is not run
    return {
        'boosting_type': 'gbdt', 'num_leaves': 31, 'learning_rate': 0.05,
        'n_estimators': 200, 'feature_fraction': 0.8, 'bagging_fraction': 0.8,
        'bagging_freq': 5, 'min_child_samples': 20, 'reg_alpha': 0.01,
        'reg_lambda': 0.01, 'random_state': 42, 'verbose': -1, 'n_jobs': -1,
        'class_weight': 'balanced',
        'min_gain_to_split': 0.0 # Default LightGBM value
    }

def enhanced_patchtst_finetune(log_returns, context_length=512, prediction_length=5, min_sequences=20):
    if not isinstance(log_returns, np.ndarray): log_returns = np.array(log_returns)
    if log_returns.ndim > 1 : log_returns = log_returns.squeeze()

    sequence_length_needed = context_length + prediction_length
    padded_returns = log_returns.copy()

    if len(log_returns) < sequence_length_needed:
        # print(f"Insufficient raw data ({len(log_returns)} < {sequence_length_needed}). Applying intelligent padding...") # Less verbose
        mean_return = np.mean(log_returns) if len(log_returns) > 0 else 0
        std_return = np.std(log_returns) if len(log_returns) > 1 else 0.01
        std_return = max(std_return, 1e-6)
        padding_length = sequence_length_needed - len(log_returns)
        synthetic_padding = np.random.normal(mean_return, std_return, padding_length)
        padded_returns = np.concatenate([synthetic_padding, log_returns])
        # print(f"Applied padding: {padding_length} synthetic points added. New length: {len(padded_returns)}") # Less verbose

    X_sequences, y_sequences = [], []
    if len(padded_returns) >= sequence_length_needed:
        for i in range(len(padded_returns) - sequence_length_needed + 1):
            seq = padded_returns[i : i + context_length]
            target = padded_returns[i + context_length : i + sequence_length_needed]
            X_sequences.append(seq)
            y_sequences.append(target)

    if len(X_sequences) >= min_sequences:
        # print(f"Created {len(X_sequences)} training sequences for fine-tuning.") # Less verbose
        return np.array(X_sequences), np.array(y_sequences), True
    else:
        # print(f"Insufficient sequences ({len(X_sequences)} < {min_sequences}) after padding. Using pre-trained weights only.") # Less verbose
        return None, None, False

def run_patchtst_foundation_forecast(
    symbol_name: str,
    historical_data_df: pd.DataFrame,
    prediction_length: int = 5,
    model_checkpoint: str = "ibm-research/patchtst-etth1-pretrain",
    fine_tune_epochs: int = 10,
    enable_fine_tuning: bool = True,
    configured_context_length: int | None = None
):
    print(f"\n--- Forecasting for {symbol_name} using PatchTST ({model_checkpoint}) ---")
    if not torch_available or 'PatchTSTForPrediction' not in globals():
        print("Torch or PatchTST not available. Skipping.")
        return {"status": "failed", "reason": "Dependencies missing", "forecast": None}
    if 'close' not in historical_data_df.columns or historical_data_df['close'].isnull().all():
        print(f"'close' column missing or all NaN for {symbol_name}. Skipping.")
        return {"status": "failed", "reason": "'close' missing or all NaN", "forecast": None}

    df_clean = historical_data_df.dropna(subset=['close'])
    if len(df_clean) < 2:
        print(f"Not enough non-NaN close prices ({len(df_clean)}) for {symbol_name}. Skipping.")
        return {"status": "failed", "reason": "Insufficient non-NaN close prices", "forecast": None}

    try:
        # print("Loading PatchTST configuration...") # Less verbose
        base_config = PatchTSTConfig.from_pretrained(model_checkpoint)
        effective_context_length = configured_context_length if configured_context_length is not None else base_config.context_length
        min_data_for_one_sequence = effective_context_length + prediction_length + 1
        if len(df_clean) < min_data_for_one_sequence:
            print(f"Data length {len(df_clean)} too short for context {effective_context_length} + pred {prediction_length}. Adjusting context or skipping.")
            effective_context_length = max(10, len(df_clean) - prediction_length - 5)
            if effective_context_length < 10 :
                return {"status": "failed", "reason": f"Cannot determine valid context with data {len(df_clean)}", "forecast": None}

        financial_config = PatchTSTConfig(
            context_length=effective_context_length, prediction_length=prediction_length,
            patch_length=min(16, effective_context_length // 2 if effective_context_length > 32 else 8),
            patch_stride=min(8, effective_context_length // 4 if effective_context_length > 32 else 4),
            num_input_channels=1, d_model=base_config.d_model,
            num_attention_heads=base_config.num_attention_heads, num_hidden_layers=base_config.num_hidden_layers,
            ffn_dim=base_config.ffn_dim, dropout=0.1, head_dropout=0.1, scaling="std", loss="mse"
        )
        # print(f"Financial config - Context: {financial_config.context_length}, Pred: {financial_config.prediction_length}, Patch: {financial_config.patch_length}") # Less verbose
        model = PatchTSTForPrediction(financial_config)

        # print("Loading and transferring compatible weights...") # Less verbose
        try:
            pretrained_model = PatchTSTForPrediction.from_pretrained(model_checkpoint, local_files_only=False, trust_remote_code=True)
            pretrained_dict = pretrained_model.state_dict()
            model_dict = model.state_dict()
            compatible_weights = {}
            for k, v in pretrained_dict.items():
                if k in model_dict and v.size() == model_dict[k].size():
                    if not any(skip_layer in k for skip_layer in ['input_embedding', 'projection', 'head', 'value_embedding', 'patch_embedding.weight', 'patch_embedding.bias']):
                        compatible_weights[k] = v
            model.load_state_dict(compatible_weights, strict=False)
            # print(f"Transferred {len(compatible_weights)}/{len(model_dict)} compatible weights") # Less verbose
        except Exception as e_load:
            print(f"Weight transfer warning/error: {e_load}. Model may use more random init for some layers.")

        device = "cuda" if torch.cuda.is_available() else "cpu"; model.to(device)
        close_prices = df_clean['close'].values.astype(np.float32)
        log_prices = np.log(np.maximum(close_prices, 1e-6))
        log_returns = np.diff(log_prices)
        if len(log_returns) < financial_config.context_length + financial_config.prediction_length:
            print(f"Insufficient log_returns ({len(log_returns)}) after diff for context/pred. Skipping fine-tune/forecast.")
            return {"status": "failed", "reason": "insufficient log_returns data", "forecast": None}

        finetuned_this_run = False
        if enable_fine_tuning and fine_tune_epochs > 0:
            # print(f"Attempting fine-tuning on {symbol_name} for {fine_tune_epochs} epochs...") # Less verbose
            X_seqs, y_seqs, finetune_data_ok = enhanced_patchtst_finetune(
                log_returns, financial_config.context_length, financial_config.prediction_length)
            if finetune_data_ok and X_seqs is not None and len(X_seqs) > 0:
                finetuned_this_run = True
                train_inputs = torch.tensor(X_seqs, dtype=torch.float32).unsqueeze(-1).to(device)
                train_targets = torch.tensor(y_seqs, dtype=torch.float32).unsqueeze(-1).to(device)
                dataset = torch.utils.data.TensorDataset(train_inputs, train_targets)
                batch_size = min(16, len(X_seqs) // 2 if len(X_seqs) >= 4 else 1)
                if batch_size == 0 and len(X_seqs) > 0: batch_size = 1

                if batch_size > 0:
                    train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True,
                                                               drop_last=True if len(X_seqs) > batch_size else False)
                    model.train(); optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)
                    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=fine_tune_epochs)
                    best_loss = float('inf'); patience, patience_counter = 3, 0

                    for epoch in range(fine_tune_epochs):
                        epoch_loss, num_batches = 0, 0
                        if not train_loader: break
                        for batch_inputs_data, batch_targets_data in train_loader:
                            optimizer.zero_grad()
                            outputs = model(past_values=batch_inputs_data, future_values=batch_targets_data)
                            loss = outputs.loss
                            loss.backward(); torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0); optimizer.step()
                            epoch_loss += loss.item(); num_batches += 1
                        if num_batches > 0:
                            avg_loss = epoch_loss / num_batches; scheduler.step()
                            print(f"Epoch {epoch+1}/{fine_tune_epochs} | Loss: {avg_loss:.6f} | LR: {scheduler.get_last_lr()[0]:.6f}")
                            if avg_loss < best_loss: best_loss = avg_loss; patience_counter = 0
                            else:
                                patience_counter += 1
                                if patience_counter >= patience: print(f"Early stopping at epoch {epoch+1}"); break
                        else: print(f"Epoch {epoch+1}/{fine_tune_epochs} | No batches. Stopping fine-tuning."); break
                    loss_display = f"{best_loss:.6f}" if best_loss != float('inf') else "N/A"
                    if num_batches > 0: print(f"Fine-tuning completed. Best loss: {loss_display}")
                else: print(f"Fine-tuning skipped for {symbol_name}: Not enough sequences or batch_size issue."); finetuned_this_run = False
            # else: print(f"Fine-tuning skipped for {symbol_name} due to insufficient sequences from enhanced_patchtst_finetune.") # Less verbose
        # else: print(f"Fine-tuning disabled or fine_tune_epochs is 0 for {symbol_name}.") # Less verbose

        model.eval()
        current_model_context_length = model.config.context_length
        if len(log_returns) < current_model_context_length:
            # print(f"Warning: log_returns ({len(log_returns)}) shorter than model context ({current_model_context_length}) for forecast. Padding...") # Less verbose
            mean_lr = np.mean(log_returns) if len(log_returns) > 0 else 0
            padding_needed = current_model_context_length - len(log_returns)
            past_returns_for_forecast = np.concatenate([np.full(padding_needed, mean_lr), log_returns])
        else:
            past_returns_for_forecast = log_returns[-current_model_context_length:]

        returns_mean = np.mean(past_returns_for_forecast); returns_std = max(np.std(past_returns_for_forecast), 1e-8)
        norm_returns_forecast_input = (past_returns_for_forecast - returns_mean) / returns_std
        past_tensor = torch.tensor(norm_returns_forecast_input, dtype=torch.float32).view(1, current_model_context_length, 1).to(device)

        with torch.no_grad(): outputs = model(past_values=past_tensor)
        fc_returns_norm = outputs.prediction_outputs.cpu().numpy().squeeze()
        if fc_returns_norm.ndim == 0: fc_returns_norm = np.array([fc_returns_norm])
        elif fc_returns_norm.ndim > 1: fc_returns_norm = fc_returns_norm.flatten()

        target_pred_len = model.config.prediction_length
        if len(fc_returns_norm) < target_pred_len:
            last_val = fc_returns_norm[-1] if len(fc_returns_norm) > 0 else 0
            fc_returns_norm = np.concatenate([fc_returns_norm, np.full(target_pred_len - len(fc_returns_norm), last_val)])
        forecast_log_returns = (fc_returns_norm[:target_pred_len] * returns_std) + returns_mean

        last_log_price = log_prices[-1]
        forecast_log_prices = last_log_price + np.cumsum(forecast_log_returns)
        forecast_prices = np.exp(forecast_log_prices)
        forecast_prices = np.maximum(forecast_prices, 0.01).tolist()

        last_actual_price = close_prices[-1]
        price_change = forecast_prices[-1] - last_actual_price
        magnitude_pct = (price_change / last_actual_price) * 100 if last_actual_price != 0 else 0
        direction = "📈 UP" if price_change > 0.001 * last_actual_price else "📉 DOWN" if price_change < -0.001 * last_actual_price else "횡보 HOLD"
        atr_val = np.nan
        if 'ATR_14' in df_clean.columns and not df_clean['ATR_14'].empty: atr_val = df_clean['ATR_14'].iloc[-1]

        return {
            "status": "success", "forecast": forecast_prices, "last_price": float(last_actual_price),
            "direction": direction, "magnitude": float(magnitude_pct), "confidence": "🟡 Medium",
            "atr_threshold": float(atr_val) if pd.notna(atr_val) else None,
            "method": f"PatchTST {'Fine-tuned' if finetuned_this_run else 'Pre-trained'} ({fine_tune_epochs} epochs attempted)",
            "model_info": {"context_length": model.config.context_length, "prediction_length": model.config.prediction_length, "fine_tuned_actually": finetuned_this_run}
        }
    except Exception as e:
        print(f"Error in PatchTST forecasting for {symbol_name}: {e}")
        traceback.print_exc()
        return {"status": "failed", "reason": f"PatchTST error: {str(e)}", "forecast": None}

def simple_ma_fallback(prices, length=5):
    if not isinstance(prices, np.ndarray): prices = np.array(prices)
    if len(prices) == 0: return np.full(length, np.nan)
    if len(prices) < 5: return np.full(length, prices[-1])
    ma_val = np.mean(prices[-5:])
    return np.full(length, ma_val)

# --- MAIN WORKFLOW FUNCTION ---
def run_full_workflow(symbol=DEFAULT_SYMBOL, start_date_str=START_DATE, end_date_str=END_DATE,
                      api_key_val=API_KEY, force_train_autoformer=False,
                      run_optuna_lgbm=False, use_foundation_model=True):
    print(f"\n{'='*40}\n🚀 ENHANCED WORKFLOW FOR: {symbol}\n{'='*40}")
    default_return = {
        "symbol": symbol, "status": "Workflow Started", "raw_data_shape": (0,0),
        "featured_data_shape": (0,0), "X_train_shape": (0,0), "X_test_shape": (0,0),
        "selected_features_count": 0, "selected_feature_names": [], "scaler_object": None,
        "ml_model_object": None, "lgbm_feature_importance": None,
        "causal_model_object": None, "causal_feature_ranking": [],
        "onnx_model_path": None,
        "forecasting_results": {
            "configured_context_length": 512, # Default, will be updated
            "patchtst_forecast": None,
            "autoformer_forecast": None
        },
        "lgbm_optimized_params": None
    }
    price_c, target_col = 'close', 'target'

    # Data Fetching
    # print(f"DEBUG: About to fetch data for {symbol} using API key ending with '...{api_key_val[-4:] if len(api_key_val)>4 else api_key_val}'") # Less verbose
    df_raw = fetch_twelve_data(symbol, api_key_val, start_date_str=start_date_str, end_date_str=end_date_str)
    if df_raw is None or df_raw.empty:
        default_return["status"] = "Data Fetching Failed"
        print(f"Workflow aborted for {symbol}: Data Fetching Failed.")
        return default_return
    default_return["raw_data_shape"] = df_raw.shape
    # print(f"DEBUG: Data fetched for {symbol}, shape: {df_raw.shape}") # Less verbose

    # Configure Context Windows
    # print(f"\n--- ⚙️ Configuring Context Windows for {symbol} ---") # Less verbose
    dynamic_context_length = 512
    if not df_raw.empty:
        context_configs = configure_extended_context(data_length=len(df_raw))
        dynamic_context_length = context_configs.get('adaptive', 512)
        # print(f"Dynamically selected context length for {symbol}: {dynamic_context_length} days") # Less verbose
    default_return["forecasting_results"]["configured_context_length"] = dynamic_context_length

    # Feature Engineering
    print(f"\n--- 🔧 Feature Engineering: {symbol} ---")
    df_f = df_raw.copy()
    df_f = add_technical_indicators(df_f)
    df_f = add_optimized_features(df_f, price_col=price_c, volume_col='volume')
    df_f = add_wavelet_features(df_f, column=price_c)
    df_f = add_entropy_features(df_f, column=price_c, window=40)
    df_f = add_advanced_technical_features(df_f, price_col=price_c, high_col='high', low_col='low', volume_col='volume')
    df_f = add_transformer_features_conceptual(df_f, column=price_c, sequence_length=20)
    df_f = add_multitimeframe_features(df_f, price_col=price_c)

    if 'log_returns' in df_f.columns:
        df_f = add_regime_features(df_f, returns_col='log_returns', price_col=price_c)
    else:
        print(f"Skipping regime features for {symbol} as 'log_returns' column is missing.")

    if 'RSI_14' in df_f.columns and 'ADX_14' in df_f.columns:
        df_f['RSI_ADX_interaction'] = df_f['RSI_14'] * df_f['ADX_14'] / 100.0
    if 'ATR_14' in df_f.columns and 'volatility_20' in df_f.columns:
        volatility_safe = df_f['volatility_20'].replace(0, np.nan)
        df_f['ATR_vol_ratio'] = df_f['ATR_14'] / volatility_safe
    default_return["featured_data_shape"] = df_f.shape
    # print(f"DEBUG: Feature engineering complete for {symbol}, shape: {df_f.shape}") # Less verbose

    # Data Cleaning Post-Feature Engineering
    print(f"\n--- Data Cleaning (Inf/NaN Handling & Imputation): {symbol} ---")
    df_f.replace([np.inf, -np.inf], np.nan, inplace=True)
    numeric_cols_to_impute = df_f.select_dtypes(include=np.number).columns
    if not numeric_cols_to_impute.empty:
        nan_counts_before = df_f[numeric_cols_to_impute].isnull().sum().sum()
        if nan_counts_before > 0:
            print(f"NaNs before imputation: {nan_counts_before}")
            df_f[numeric_cols_to_impute] = df_f[numeric_cols_to_impute].interpolate(method='linear', limit_direction='both', axis=0)
            df_f[numeric_cols_to_impute] = df_f[numeric_cols_to_impute].fillna(method='bfill').fillna(method='ffill').fillna(0)
            print(f"NaNs after imputation: {df_f[numeric_cols_to_impute].isnull().sum().sum()}")
        # else: print("No NaNs in numeric features to impute.") # Less verbose
    # else: print("No numeric columns for imputation.") # Less verbose
    # print(f"DEBUG: Data cleaning complete for {symbol}.") # Less verbose

    # Regime Detection and Target Definition
    print(f"\n--- 📊 Regime Detection (Simplified): {symbol} ---")
    df_f = detect_regimes_simple(df_f, column=price_c)
    print(f"\n--- 🎯 Target Definition: {symbol} ---")
    df_f = balanced_target_definition(df_f, column=price_c, periods=5)
    # print(f"DEBUG: Target definition complete for {symbol}.") # Less verbose

    # Causal Discovery (Conditional)
    causal_model_obj = None
    causal_feat_ranking = []
    if not SKIP_CAUSAL_ANALYSIS_FOR_DEBUGGING:
        print(f"\n--- Causal Discovery & Ranking (Actual Process): {symbol} ---")
        if df_f is not None and not df_f.empty and target_col in df_f.columns and df_f[target_col].nunique(dropna=True) > 1:
            causal_model_obj = discover_causal_structure(df_f.copy(), target_col=target_col, price_c=price_c, symbol=symbol, max_feats=10)
            if causal_model_obj:
                print(f"Causal model object created for {symbol}.")
                # causal_feat_ranking = causal_feature_selection(causal_model_obj, df_f.copy(), target_col=target_col, max_causal_feats=10) # Placeholder
            else:
                print(f"Causal model object was not created for {symbol}, skipping feature ranking.")
        else:
            print(f"Skipping Causal Discovery for {symbol} due to data/target issues.")
    else:
        print(f"\n--- SKIPPING Causal Discovery & Ranking for {symbol} (DEBUG MODE) ---")
    default_return["causal_model_object"] = causal_model_obj
    default_return["causal_feature_ranking"] = causal_feat_ranking
    # print(f"DEBUG: Causal discovery section complete for {symbol}.") # Less verbose

    # ML Preparation and Feature Selection
    print(f"\n--- ML Preparation & Feature Selection: {symbol} ---")
    X_tr, X_te, y_tr, y_te, scaler_obj = None, None, None, None, None
    sel_feat_names = []
    ml_model = None
    lgbm_feat_imp_df = None
    onnx_file_path = None
    current_status_ml = "ML Prep Incomplete"

    if df_f is None or df_f.empty or target_col not in df_f.columns or df_f[target_col].isnull().all():
        current_status_ml = "ML Prep Failed - DataFrame empty, target missing, or target all NaN"
        print(f"{current_status_ml} for {symbol}.")
    elif df_f[target_col].nunique(dropna=True) <= 1:
        unique_vals_count = df_f[target_col].nunique(dropna=True)
        current_status_ml = f"ML Prep Skipped - Target has {unique_vals_count} unique non-NaN value(s). Training not meaningful."
        print(f"{current_status_ml} for {symbol}.")
    else:
        ml_data_prep_output = prepare_ml_data(df_f.copy(), target_col=target_col, test_split_size=0.15, min_test_samples=30)

        if ml_data_prep_output is None or not all(item is not None for item in ml_data_prep_output[:4]):
            current_status_ml = "ML Data Preparation Failed or returned insufficient data."
            print(f"{current_status_ml} for {symbol}. Skipping subsequent ML steps.")
        else:
            X_tr, X_te, y_tr, y_te, scaler_obj = ml_data_prep_output
            default_return["scaler_object"] = scaler_obj
            default_return["X_train_shape"] = X_tr.shape if X_tr is not None else (0,0)
            default_return["X_test_shape"] = X_te.shape if X_te is not None else (0,0)

            if X_tr is None or X_tr.empty or y_tr is None or y_tr.empty:
                current_status_ml = "ML Training Data (X_tr or y_tr) is empty after preparation. Skipping training."
                print(f"{current_status_ml} for {symbol}.")
            else:
                # print("Starting feature selection...") # Less verbose
                selected_features_df = prioritized_feature_selection(X_tr.copy(), y_tr.copy(), causal_feat_ranking, max_features=35)

                if selected_features_df is not None and 'Feature' in selected_features_df.columns and not selected_features_df.empty:
                    sel_feat_names = selected_features_df['Feature'].tolist()
                    if not sel_feat_names:
                         print("Prioritized selection returned no features. Using simple variance fallback on original X_tr.")
                         sel_feat_df_fallback = simple_feature_selection_fallback(X_tr.copy(), y_tr.copy(), max_features=20)
                         if sel_feat_df_fallback is not None and 'Feature' in sel_feat_df_fallback.columns:
                            sel_feat_names = sel_feat_df_fallback['Feature'].tolist()
                         else:
                            sel_feat_names = X_tr.columns[:20].tolist() if not X_tr.empty else []
                else:
                    print("Prioritized feature selection failed or returned empty DataFrame. Using simple variance fallback.")
                    sel_feat_df_fallback = simple_feature_selection_fallback(X_tr.copy(), y_tr.copy(), max_features=20)
                    if sel_feat_df_fallback is not None and 'Feature' in sel_feat_df_fallback.columns:
                        sel_feat_names = sel_feat_df_fallback['Feature'].tolist()
                    else:
                        sel_feat_names = X_tr.columns[:20].tolist() if not X_tr.empty else []


                default_return["selected_feature_names"] = sel_feat_names
                default_return["selected_features_count"] = len(sel_feat_names)
                print(f"Selected {len(sel_feat_names)} features: {sel_feat_names[:10]}...")

                if not sel_feat_names:
                    current_status_ml = "No features selected. Skipping LightGBM training."
                    print(current_status_ml)
                else:
                    X_tr_selected = X_tr[sel_feat_names].copy()
                    X_te_selected = X_te[sel_feat_names].copy() if X_te is not None and not X_te.empty and all(f in X_te.columns for f in sel_feat_names) else pd.DataFrame()


                    # Prepare base_params for Optuna, ensuring num_class is set correctly
                    num_classes_for_optuna = y_tr.nunique()
                    optuna_base_params = optimized_lightgbm_params() # Start with general defaults
                    optuna_base_params['objective'] = 'multiclass' if num_classes_for_optuna > 2 else 'binary'
                    optuna_base_params['metric'] = 'multi_logloss' if num_classes_for_optuna > 2 else 'binary_logloss'
                    if num_classes_for_optuna > 2:
                        optuna_base_params['num_class'] = num_classes_for_optuna
                    elif 'num_class' in optuna_base_params: # remove if binary
                        del optuna_base_params['num_class']

                    lgbm_final_params = optuna_base_params.copy() # These will be the params used for training

                    if run_optuna_lgbm and optuna_available:
                        print(f"\n--- Hyperparameter Optimization (Optuna for LightGBM): {symbol} ---")
                        # Pass the correctly configured optuna_base_params to Optuna
                        tuned_params_from_optuna = optimize_lgbm_hyperparameters(X_tr_selected.copy(), y_tr.copy(), optuna_base_params, n_trials=50) # Increased trials
                        if tuned_params_from_optuna:
                           lgbm_final_params.update(tuned_params_from_optuna) # Update with Optuna's best
                        default_return["lgbm_optimized_params"] = lgbm_final_params
                    else:
                         print("Optuna HPO for LightGBM skipped or Optuna not available.")
                         default_return["lgbm_optimized_params"] = lgbm_final_params # Store base/default if Optuna not run


                    print(f"\n--- LightGBM Model Training: {symbol} ---")
                    # Pass the final determined parameters to the training function
                    ml_model, lgbm_feat_imp_df = train_lightgbm_model(X_tr_selected, y_tr, X_te_selected, y_te, optimized_params=lgbm_final_params)
                    default_return["ml_model_object"] = ml_model
                    default_return["lgbm_feature_importance"] = lgbm_feat_imp_df

                    if ml_model:
                        current_status_ml = "LightGBM Model Trained"
                        plot_feature_importance(lgbm_feat_imp_df, top_n=20, symbol_for_plot=symbol)
                        if not X_tr_selected.empty:
                             onnx_file_path = export_lgbm_to_onnx(ml_model, X_tr_selected.head(1), file_path=f"lgbm_model_{symbol}.onnx")
                             default_return["onnx_model_path"] = onnx_file_path
                        else:
                            print("X_tr_selected is empty. Skipping ONNX export.")
                    else:
                        current_status_ml = "LightGBM Model Training Failed."
                        print(current_status_ml)
    default_return["status"] = current_status_ml
    # print(f"DEBUG: ML section complete for {symbol}. Status: {current_status_ml}") # Less verbose


    # Autoformer Forecasting (Optional)
    autoformer_predictions = None
    if torch_available and 'AutoformerPredictor' in globals():
        # print(f"\n--- Autoformer Forecasting (Custom): {symbol} ---") # Less verbose
        if price_c in df_raw.columns and not df_raw[price_c].isnull().all():
            series_for_autoformer = df_raw[price_c].copy()
            autoformer_input_len = min(90, max(10, len(series_for_autoformer) - 15))
            autoformer_pred_len = 5

            if autoformer_input_len > 0 and len(series_for_autoformer) > autoformer_input_len + autoformer_pred_len :
                autoformer_model = train_or_load_autoformer(series_for_autoformer,
                                                            input_len=autoformer_input_len,
                                                            pred_len=autoformer_pred_len,
                                                            force_train=force_train_autoformer)
                if autoformer_model and autoformer_model.model is not None:
                    autoformer_predictions = autoformer_model.predict(series_for_autoformer)
                    print(f"Autoformer forecast for {symbol}: {autoformer_predictions}")
                    default_return["forecasting_results"]["autoformer_forecast"] = autoformer_predictions.tolist() if autoformer_predictions is not None else None
                # else: print(f"Autoformer model not available/trained for {symbol}.") # Less verbose
            # else: print(f"Not enough data for Autoformer input/pred length on {symbol} (Data len: {len(series_for_autoformer)}, Needed: >{autoformer_input_len + autoformer_pred_len}).") # Less verbose
        # else: print(f"Price column '{price_c}' missing or all NaNs in df_raw for Autoformer on {symbol}.") # Less verbose
    # else: print(f"PyTorch or AutoformerPredictor not available. Skipping Autoformer for {symbol}.") # Less verbose
    # print(f"DEBUG: Autoformer section complete for {symbol}.") # Less verbose


    # Foundation Model (PatchTST) Forecasting (Optional)
    if use_foundation_model and torch_available and 'PatchTSTForPrediction' in globals():
        print(f"\n--- Foundation Model Forecasting (PatchTST): {symbol} ---")
        patchtst_results = run_patchtst_foundation_forecast(
            symbol_name=symbol,
            historical_data_df=df_raw.copy(),
            prediction_length=5,
            enable_fine_tuning=True,
            fine_tune_epochs=10, # Increased epochs for PatchTST fine-tuning
            configured_context_length=dynamic_context_length
        )
        default_return["forecasting_results"]["patchtst_forecast"] = patchtst_results
        if patchtst_results and patchtst_results.get("status") == "success":
            print(f"PatchTST Forecast for {symbol}: {patchtst_results.get('forecast')}")
            print(f"  Direction: {patchtst_results.get('direction')}, Magnitude: {patchtst_results.get('magnitude',0):.2f}%")
    else:
        # print(f"PatchTST forecasting skipped for {symbol} (disabled or dependencies missing).") # Less verbose
        default_return["forecasting_results"]["patchtst_forecast"] = {"status": "skipped", "reason": "Disabled or dependencies missing"}
    # print(f"DEBUG: PatchTST section complete for {symbol}.") # Less verbose

    print(f"\n🏁 Workflow completed for {symbol}. Final Status: {default_return['status']}")
    return default_return

# --- Example Usage and Main Execution ---
if __name__ == "__main__":
    print("DEBUG: Script execution started, entering __main__ block.")
    start_time = time.time()
    
    # Ensure API_KEY is set correctly.
    # For example, you could set it from an environment variable:
    # API_KEY = os.environ.get("TWELVE_DATA_API_KEY", "YOUR_API_KEY_HERE")
    # Or directly in the script for testing (replace "YOUR_API_KEY_HERE"):
    # API_KEY = "your_real_api_key_goes_here" 
    
    # This line should be MODIFIED LOCALLY with your actual key.
    # It's inheriting from TWELVE_DATA_API_KEY which is also a placeholder.
    # For this run, I will manually set it here if it's the placeholder.
    
    current_api_key = API_KEY # Uses the global API_KEY

    if current_api_key == "YOUR_API_KEY_HERE" or not current_api_key:
        print("🛑 CRITICAL: TWELVE_DATA_API_KEY is not set. Update 'YOUR_API_KEY_HERE' in the script constants or set API_KEY directly.")
        print("🛑 CRITICAL: Workflow cannot proceed without a valid API key.")
        # exit() # Uncomment to force exit
    else:
        print(f"DEBUG: API_KEY appears to be set. Proceeding. Key ends with '...{current_api_key[-4:] if len(current_api_key)>4 else current_api_key}'")

    symbols_to_run = ["AAPL", "GOOGL"]
    # symbols_to_run = ["MSFT"] # For a single quick test
    print(f"DEBUG: Symbols to process: {symbols_to_run}")
    all_results_dict = {}

    if current_api_key == "YOUR_API_KEY_HERE" or not current_api_key:
        print("DEBUG: Halting before loop due to missing API Key.")
    else:
        for sym_item in symbols_to_run:
            print(f"DEBUG: Starting main loop for symbol: {sym_item}")
            try:
                workflow_output = run_full_workflow(
                    symbol=sym_item,
                    api_key_val=current_api_key, 
                    run_optuna_lgbm=True, # <<<< MODIFIED TO TRUE
                    force_train_autoformer=False, 
                    use_foundation_model=True 
                )
                all_results_dict[sym_item] = workflow_output
                print(f"\n--- Results Summary for {sym_item} ---")
                if workflow_output:
                    print(f"  Overall Status: {workflow_output.get('status')}")
                    print(f"  Raw Data Shape: {workflow_output.get('raw_data_shape')}")
                    print(f"  Featured Data Shape: {workflow_output.get('featured_data_shape')}")
                    print(f"  Selected Features Count: {workflow_output.get('selected_features_count')}")
                    
                    lgbm_feat_imp = workflow_output.get("lgbm_feature_importance")
                    if lgbm_feat_imp is not None and not lgbm_feat_imp.empty:
                        print(f"  Top LGBM Features: {lgbm_feat_imp['Feature'].head(3).tolist()}")
                    
                    forecast_summary = workflow_output.get("forecasting_results", {})
                    patchtst_info = forecast_summary.get("patchtst_forecast")
                    autoformer_info = forecast_summary.get("autoformer_forecast")

                    if patchtst_info and isinstance(patchtst_info, dict) and patchtst_info.get("status") == "success":
                        print(f"  PatchTST Forecast ({patchtst_info.get('method', 'N/A')}):")
                        print(f"    Values: {patchtst_info.get('forecast')}")
                        print(f"    Direction: {patchtst_info.get('direction')}, Magnitude: {patchtst_info.get('magnitude', 0):.2f}%")
                    elif patchtst_info and isinstance(patchtst_info, dict):
                        print(f"  PatchTST Status: {patchtst_info.get('status')}, Reason: {patchtst_info.get('reason')}")
                    
                    if autoformer_info:
                         print(f"  Autoformer Forecast: {autoformer_info}")
                    
                    print(f"  ONNX Model Path: {workflow_output.get('onnx_model_path')}")
                print("-" * 40)
                # print(f"DEBUG: Completed processing for symbol: {sym_item}") # Less verbose

            except Exception as e_main_loop:
                print(f"🛑 ERROR: Unhandled exception in main loop for symbol {sym_item}: {e_main_loop}")
                traceback.print_exc()
                all_results_dict[sym_item] = {"status": f"Error: {e_main_loop}", "symbol": sym_item}
                # print(f"DEBUG: Error processing symbol: {sym_item}") # Less verbose


    end_time = time.time()
    print(f"\nTotal execution time for {len(symbols_to_run)} symbol(s): {(end_time - start_time):.2f} seconds.")
    print("DEBUG: Script __main__ block finished.")



Optuna imported successfully.
PyTorch imported successfully.
PyTorch CUDA available: True, Version: 12.1
Using PyTorch on GPU: NVIDIA GeForce RTX 4070 Laptop GPU
DoWhy 0.10 and NetworkX 3.1 imported successfully.
ONNX, ONNXRuntime, skl2onnx, and onnxmltools imported successfully.
Onnxmltools version: 1.11.1

All libraries and modules conditional imports attempted.
DEBUG: Script execution started, entering __main__ block.
DEBUG: API_KEY appears to be set. Proceeding. Key ends with '...ef0a'
DEBUG: Symbols to process: ['AAPL', 'GOOGL']
DEBUG: Starting main loop for symbol: AAPL

🚀 ENHANCED WORKFLOW FOR: AAPL
Fetching data for AAPL from Twelve Data (interval=1day, from 2022-06-04 to 2025-06-03)...
Successfully fetched/processed 750 data points for AAPL.

--- 🔧 Feature Engineering: AAPL ---
Adding technical indicators...


/tmp/ipykernel_35088/1678251532.py:355: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[9.98611407e+09 8.00782855e+09 1.23548189e+10 1.76871775e+10
 1.09688179e+10 9.95967086e+09 9.95940605e+09 1.25755687e+10
 9.97233035e+09 9.79635737e+09 1.02719769e+10 1.05694694e+10
 9.63284651e+09 9.45818898e+09 1.13631385e+10 1.14780874e+10
 1.14194116e+10 1.24235717e+10 9.87757346e+09 1.00419950e+10
 1.03075722e+10 1.22197531e+10 1.27330542e+10 1.64772630e+10
 1.09881365e+10 1.35716587e+10 9.19816034e+09 9.98346202e+09
 1.18232614e+10 9.67097850e+09 1.16499329e+10 9.33857157e+09
 9.74196305e+09 1.38739808e+10 8.68182651e+09 1.20859660e+10
 1.35908341e+10 1.06564850e+10 1.70375068e+10 1.24421969e+10
 1.68017624e+10 1.41309629e+10 1.28473936e+10 1.61259518e+10
 1.27816217e+10 1.15707123e+10 9.99105288e+09 1.58872146e+10
 1.24359106e+10 1.20936924e+10 1.42446076e+10 9.28966227e+09
 1.26294602e+10 1.12878711e+10 1.13142


--- Data Cleaning (Inf/NaN Handling & Imputation): AAPL ---
NaNs before imputation: 1858
NaNs after imputation: 0

--- 📊 Regime Detection (Simplified): AAPL ---
Simple Regimes (0:Med,1:Low,2:High):
regime_simple
0    34.666667
1    32.666667
2    32.666667
Name: proportion, dtype: float64 %

--- 🎯 Target Definition: AAPL ---
Target distribution:
target
0    55.333333
1    44.666667
Name: proportion, dtype: float64

--- SKIPPING Causal Discovery & Ranking for AAPL (DEBUG MODE) ---

--- ML Preparation & Feature Selection: AAPL ---
Train shapes: X_train=(638, 100), y_train=(638,); Test shapes: X_test=(112, 100), y_test=(112,)
Prioritized feature selection: Causal Ranking + Mutual Information...
No valid causal ranking provided or num_causal_to_select is 0.
Selected 35 features: ['MACDh_12_26_9', 'SMA_10', 'SMA_20', 'EMA_20', 'SMA_50', 'EMA_50', 'SMA_100', 'EMA_100', 'SMA_200', 'EMA_200']...


/tmp/ipykernel_35088/1678251532.py:579: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_reg['regime_simple'] = df_reg['regime_simple'].fillna(method='bfill').fillna(method='ffill').fillna(0)
[I 2025-06-03 12:12:31,775] A new study created in memory with name: no-name-5351b1f5-4972-41a4-9615-2ec8cadf8628



--- Hyperparameter Optimization (Optuna for LightGBM): AAPL ---
Optimizing LightGBM HPs with Optuna (50 trials)...


...

[LightGBM] [Warning] bagging_fraction is set=0.6617438762026049, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6617438762026049
[LightGBM] [Warning] feature_fraction is set=0.9217247282929125, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9217247282929125
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] min_gain_to_split is set=0.04819052063531404, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.04819052063531404
[I 2025-06-03 12:12:38,313] Trial 0 finished with value: 0.6621338281221715 and parameters: {'n_estimators': 500, 'learning_rate': 0.08049659454612014, 'num_leaves': 44, 'max_depth': 3, 'min_child_samples': 36, 'feature_fraction': 0.9217247282929125, 'bagging_fraction': 0.6617438762026049, 'bagging_freq': 4, 'reg_alpha': 0.026630791527572355, 'reg_lambda': 0.08346765548141377, 'min_gain_to_split': 0.04819052063531404}. Best is tria

/tmp/ipykernel_35088/1678251532.py:938: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  ax = sns.barplot(x='Plot_Importance', y='Feature', data=plot_data, palette="viridis", orient='h')
The maximum opset needed by this model is only 9.


Feature importance plot saved to feature_importance_GOOGL.png

Exporting LGBM model to ONNX: lgbm_model_GOOGL.onnx (opset=12)
Model exported to ONNX: lgbm_model_GOOGL.onnx
ONNX model check OK.
Autoformer forecast for GOOGL: [172.77906926 171.45177371 172.60277009 171.99249383 172.54494512]

--- Foundation Model Forecasting (PatchTST): GOOGL ---

--- Forecasting for GOOGL using PatchTST (ibm-research/patchtst-etth1-pretrain) ---


/home/chaitanyakharche/Desktop/stock/venv_py310/lib/python3.10/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Weight transfer warning/error: Due to a serious vulnerability issue in `torch.load`, even with `weights_only=True`, we now require users to upgrade torch to at least v2.6 in order to use the function. This version restriction does not apply when loading files with safetensors.
See the vulnerability report here https://nvd.nist.gov/vuln/detail/CVE-2025-32434. Model may use more random init for some layers.
Epoch 1/10 | Loss: 0.000429 | LR: 0.000098
Epoch 2/10 | Loss: 0.000427 | LR: 0.000090
Epoch 3/10 | Loss: 0.000426 | LR: 0.000079
Epoch 4/10 | Loss: 0.000425 | LR: 0.000065
Epoch 5/10 | Loss: 0.000416 | LR: 0.000050
Epoch 6/10 | Loss: 0.000404 | LR: 0.000035
Epoch 7/10 | Loss: 0.000415 | LR: 0.000021
Epoch 8/10 | Loss: 0.000420 | LR: 0.000010
Epoch 9/10 | Loss: 0.000410 | LR: 0.000002
Early stopping at epoch 9
Fine-tuning completed. Best loss: 0.000404
PatchTST Forecast for GOOGL: [168.36952209472656, 168.96121215820312, 170.31472778320312, 171.6829376220703, 172.35374450683594]
  Dire